In [97]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle

In [98]:
# Read files

df = pd.read_parquet("out/qc/qc_summary_df.parquet")
with open("out/qc/issue_examples.pickle", "rb") as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    issue_trips = pickle.load(f)

In [99]:
df.head()

,pid,avg_speed_trips,na_times_trips,same_time_trips,min_max_time_issue_trips,very_long_trips
0,19400,0,0,0,0,0


In [100]:
issue_trips

[{'pid': '19400',
  'avg_speed_trips': [],
  'na_times_trips': [],
  'same_time_trips': [],
  'min_max_time_issue_trips': [],
  'very_long_trips': []}]

In [101]:
df1 = gpd.read_parquet("out/trips/test_trips_19400_full.parquet")
df1

,seg_combined,typ,bus_stop_time,time_diff_seconds,stop_dist,ping_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,time
0,0.0,S,2023-04-14 12:53:28.999471936,333.722642,470.955899,1144.333112,3.156813,54B19400.02387861528835805613792023-04-14,18295,19400-18295,POINT (349929.59037 565172.64236),NaT,2023-04-14 12:53:28.999471936
1,8.0,S,2023-04-14 12:59:02.722113884,477.277886,673.543859,1144.333112,3.156813,54B19400.02387861528835805613792023-04-14,15935,19400-15935,POINT (349854.44094 564988.51657),NaT,2023-04-14 12:59:02.722113884
2,19.5,B,2023-04-14 13:07:00.000000000,NaN,NaN,NaN,NaN,54B19400.02387861528835805613792023-04-14,None,None,POINT (349234.69540 564993.90637),2023-04-14 13:07:00,2023-04-14 13:07:00.000000000
3,20.0,S,2023-04-14 13:07:10.640227157,380.539323,537.024512,423.276056,3.156813,54B19400.02387861528835805613792023-04-14,10368,19400-10368,POINT (349234.52984 564993.88739),NaT,2023-04-14 13:07:10.640227157
4,22.5,B,NaT,NaN,NaN,NaN,NaN,54B19400.02387861528835805613792023-04-14,None,None,POINT (349218.38475 565182.66463),2023-04-14 13:12:00,2023-04-14 13:12:00.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,55.0,S,2023-10-20 11:06:12.177530637,58.765745,324.047213,1653.914958,12.334978,54B19400.02485796938835805685492023-10-20,10389,19400-10389,POINT (349142.96394 567849.13889),NaT,2023-10-20 11:06:12.177530637
326,57.5,B,NaT,NaN,NaN,NaN,NaN,54B19400.02485796938835805685492023-10-20,None,None,POINT (349135.18506 568002.71355),2023-10-20 11:07:00,2023-10-20 11:07:00.000000000
327,58.0,S,2023-10-20 11:07:48.987580170,594.853899,741.751074,748.009305,2.789345,54B19400.02485796938835805685492023-10-20,4601,19400-4601,POINT (349138.03623 568058.03179),NaT,2023-10-20 11:07:48.987580170
328,71.5,B,2023-10-20 11:17:00.000000000,NaN,NaN,NaN,NaN,54B19400.02485796938835805685492023-10-20,None,None,POINT (349458.14026 568537.85035),2023-10-20 11:17:00,2023-10-20 11:17:00.000000000


In [102]:
df1[(df1['unique_trip_vehicle_day'] == '54B19400.02387861528835805680352023-04-14')]

,seg_combined,typ,bus_stop_time,time_diff_seconds,stop_dist,ping_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,time
125,0.0,S,2023-04-14 12:38:51.316146894,1280.251860,470.955899,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,18295,19400-18295,POINT (349929.59037 565172.64236),NaT,2023-04-14 12:38:51.316146894
126,8.0,S,2023-04-14 13:00:11.568006598,1830.964788,673.542211,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,15935,19400-15935,POINT (349854.44094 564988.51657),NaT,2023-04-14 13:00:11.568006598
127,20.0,S,2023-04-14 13:30:42.532794848,842.737378,310.010985,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,10368,19400-10368,POINT (349234.52984 564993.88739),NaT,2023-04-14 13:30:42.532794848
128,23.0,S,2023-04-14 13:44:45.270172847,630.451736,231.919182,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,16007,19400-16007,POINT (349215.23162 565296.53609),NaT,2023-04-14 13:44:45.270172847
129,26.0,S,2023-04-14 13:55:15.721909299,297.334359,109.377986,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,10370,19400-10370,POINT (349214.17425 565522.78197),NaT,2023-04-14 13:55:15.721909299
130,29.0,S,2023-04-14 14:00:13.056268651,564.765831,207.755840,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,10383,19400-10383,POINT (349212.17897 565619.40069),NaT,2023-04-14 14:00:13.056268651
131,32.0,S,2023-04-14 14:09:37.822099852,764.121583,281.091228,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,10384,19400-10384,POINT (349204.78283 565812.05939),NaT,2023-04-14 14:09:37.822099852
132,35.0,S,2023-04-14 14:22:21.943682565,2247.784464,826.874295,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,10385,19400-10385,POINT (349191.27707 566082.98179),NaT,2023-04-14 14:22:21.943682565
133,41.0,S,2023-04-14 14:59:49.728146509,352.543004,129.687144,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,10386,19400-10386,POINT (349172.47595 566906.45088),NaT,2023-04-14 14:59:49.728146509
134,43.0,S,2023-04-14 15:05:42.271150402,569.361029,209.446238,4879.013677,0.822885,54B19400.02387861528835805680352023-04-14,10387,19400-10387,POINT (349167.51063 567036.03778),NaT,2023-04-14 15:05:42.271150402


In [54]:
df = pd.read_parquet("out/pids/3910.parquet")

In [63]:
df[df['unique_trip_vehicle_day'] == '1523910.02353404888834757313912023-01-06'].sort_values('data_time')

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,scrape_file,data_time,data_hour,data_date,unique_trip_vehicle_day
244,1391,20230106 17:32,41.950325,-87.644402,334,3910.0,152,Harlem,1,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/17:32:56.json,2023-01-06 17:32:00,17,2023-01-06,1523910.02353404888834757313912023-01-06
245,1391,20230106 17:37,41.949211,-87.643794,157,3910.0,152,Harlem,2720,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/17:37:56.json,2023-01-06 17:37:00,17,2023-01-06,1523910.02353404888834757313912023-01-06
246,1391,20230106 17:42,41.947334,-87.648165,275,3910.0,152,Harlem,4483,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/17:42:56.json,2023-01-06 17:42:00,17,2023-01-06,1523910.02353404888834757313912023-01-06
247,1391,20230106 17:47,41.947140,-87.659150,262,3910.0,152,Harlem,7635,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/17:47:56.json,2023-01-06 17:47:00,17,2023-01-06,1523910.02353404888834757313912023-01-06
248,1391,20230106 17:52,41.946957,-87.672920,269,3910.0,152,Harlem,11397,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/17:52:56.json,2023-01-06 17:52:00,17,2023-01-06,1523910.02353404888834757313912023-01-06
249,1391,20230106 17:57,41.946846,-87.683372,268,3910.0,152,Harlem,14259,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/17:57:56.json,2023-01-06 17:57:00,17,2023-01-06,1523910.02353404888834757313912023-01-06
250,1391,20230106 18:02,41.946693,-87.699741,269,3910.0,152,Harlem,18703,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/18:02:56.json,2023-01-06 18:02:00,18,2023-01-06,1523910.02353404888834757313912023-01-06
251,1391,20230106 18:07,41.946676,-87.709615,265,3910.0,152,Harlem,21399,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/18:07:56.json,2023-01-06 18:07:00,18,2023-01-06,1523910.02353404888834757313912023-01-06
252,1391,20230106 18:12,41.946487,-87.724221,269,3910.0,152,Harlem,25378,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/18:12:56.json,2023-01-06 18:12:00,18,2023-01-06,1523910.02353404888834757313912023-01-06
253,1391,20230106 18:17,41.946388,-87.734160,269,3910.0,152,Harlem,28086,False,88347573,235340488,152 -416,None,bus_data/2023-01-06/18:17:56.json,2023-01-06 18:17:00,18,2023-01-06,1523910.02353404888834757313912023-01-06
